In [1]:
import numpy as np
import pandas as pd
import os
import glob
import pickle
import json
import sys
import argparse
from datetime import datetime, timedelta
from scipy import stats
from openbb_terminal.sdk import openbb

libwebkit2gtk-4.0.so.37: cannot open shared object file: No such file or directory


INFO:openbb_terminal.cryptocurrency.onchain.bitquery_model:START
INFO:openbb_terminal.cryptocurrency.onchain.bitquery_model:END


Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

PyWry is not installed or missing required linux dependencies.

Install PyWry
pip install pywry --upgrade

Platform-specific notes
Here is the underlying web engine each platform uses you might need to install.

Linux
Pywry uses gtk-rs and its related libraries for window creation and Wry also needs WebKitGTK for WebView.
To activate interactive plots/tables in pywry window, please make sure the following packages are installed:

Arch Linux / Manjaro:
sudo pacman -S webkit2gtk

Debian / Ubuntu:
sudo apt install libwebkit2gtk-4.0-dev

Fedora / CentOS / AlmaLinux:
sudo dnf install gtk3-devel webkit2gtk3-devel

Continuing with default plotly behaviour (open in browser).

INFO:openbb_terminal.loggers:Logging configuration finished
INFO:openbb_terminal.loggers:Logging set to ['file', 'posthog']
INFO:openbb_terminal.loggers:Verbosity set to 20
INFO:openbb_terminal.loggers:LOGFORMAT: %(levelname)s-%(appName)s-%(commitHash)s-%(appId)s-%(sessionId)s-%(userId)s-%(asctime)s-%(name)s-%(funcName)s-%(lineno)s-%(message)s
INFO:openbb_terminal.core.log.generation.settings_logger:STARTUP: {"PREFERENCES": {"PLOT_BACKEND": null, "PLOT_DPI": 100, "PLOT_HEIGHT": 500, "PLOT_WIDTH": 800, "PLOT_HEIGHT_PERCENTAGE": 50.0, "PLOT_WIDTH_PERCENTAGE": 70.0, "PLOT_OPEN_EXPORT": false, "PLOT_ENABLE_PYWRY": true, "PLOT_PYWRY_WIDTH": 1400, "PLOT_PYWRY_HEIGHT": 762, "FILE_OVERWRITE": false, "SHOW_VERSION": true, "RETRY_WITH_LOAD": false, "USE_TABULATE_DF": true, "USE_INTERACTIVE_DF": false, "USE_CLEAR_AFTER_CMD": false, "USE_DATETIME": true, "USE_PROMPT_TOOLKIT": true, "USE_PLOT_AUTOSCALING": false, "ENABLE_THOUGHTS_DAY": false, "ENABLE_QUICK_EXIT": false, "OPEN_REPORT_AS_HTML": true,

In [2]:
config_dict = None
config_file = f"/app/config.json"
with open(config_file) as json_file:
    config_dict = json.load(json_file)

In [ ]:
def make_targets_all_symbols_02(start_date, end_date, config_dict):
    '''
    Function to return the complete dataframe for all symbols and all annual report date periods
    '''
    symbol_names = [os.path.basename(folder) for folder in glob.glob(os.path.join(config_dict['annual_reports_pdf_save_directory'], '*')) \
                            if os.path.isdir(folder)]
    price_data_sp500 = openbb.stocks.load(
        '^GSPC', start_date=start_date, end_date=end_date, 
        verbose=False)
    full_df = pd.DataFrame()
    #Iterate over all symbols in the directory
    for i, symbol in enumerate(symbol_names):
        df = make_targets(symbol, start_date, end_date, price_data_sp500, config_dict)
        full_df = pd.concat([full_df, df], ignore_index=True)
        print('Completed: {}/{}'.format(i+1, len(symbol_names)))
    return full_df

In [4]:
openbb.__version__
start_date='2002-01-01'
end_date='2023-12-31'
price_data_sp500 = openbb.stocks.load(
        '^GSPC', start_date=start_date, end_date=end_date, 
        verbose=False)

INFO:openbb_terminal.stocks.stocks_helper:START
INFO:openbb_terminal.stocks.stocks_helper:{"INPUT": {"start_date": "2002-01-01", "interval": "1440", "end_date": "2023-12-31", "prepost": "False", "source": "YahooFinance", "weekly": "False", "monthly": "False", "verbose": "False", "symbol": "^GSPC", "chart": "False"}, "VIRTUAL_PATH": "stocks.load", "CHART": false}
INFO:openbb_terminal.stocks.stocks_helper:END


In [5]:
price_data_sp500


,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
date,,,,,,,,
2002-01-02,1148.079956,1154.670044,1136.229980,1154.670044,1154.670044,1171000000,0.0,0.0
2002-01-03,1154.670044,1165.270020,1154.010010,1165.270020,1165.270020,1398900000,0.0,0.0
2002-01-04,1165.270020,1176.550049,1163.420044,1172.510010,1172.510010,1513000000,0.0,0.0
2002-01-07,1172.510010,1176.969971,1163.550049,1164.890015,1164.890015,1308300000,0.0,0.0
2002-01-08,1164.890015,1167.599976,1157.459961,1160.709961,1160.709961,1258800000,0.0,0.0
...,...,...,...,...,...,...,...,...
2023-09-22,4341.740234,4357.399902,4316.490234,4320.060059,4320.060059,3349570000,0.0,0.0
2023-09-25,4310.620117,4338.509766,4302.700195,4337.439941,4337.439941,3195650000,0.0,0.0
2023-09-26,4312.879883,4313.009766,4265.979980,4273.529785,4273.529785,3472340000,0.0,0.0


In [ ]:

start_date='2002-01-01'
end_date='2023-12-31'
targets_df = make_targets_all_symbols(start_date, end_date, config_dict)